In [1]:
from kaggle_secrets import UserSecretsClient
token = UserSecretsClient().get_secret('HF_TOKEN')
!git config --global credential.helper store
!huggingface-cli login --token $token --add-to-git-credential

Token is valid (permission: read).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [2]:
from dataclasses import dataclass
import pandas as pd
from transformers import pipeline
from tqdm import tqdm
tqdm.pandas()

2024-06-10 12:54:46.005261: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-10 12:54:46.005406: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-10 12:54:46.137690: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
@dataclass
class Config:
    model = 'meta-llama/Meta-Llama-3-8B-Instruct'
    train_path = '/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv'
    test_path = '/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv'

In [4]:
cfg = Config()
pipe = pipeline('text-generation', cfg.model, device_map='auto')

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
def prompt_engineering(essay):
    instruction = 'Read the following essay and assign a score from 1 to 6 where 6 is the best. Output only a single number without any explanation.\n\n'
    
    prompt = [
        {'role': 'system', 'content': "As a teacher, you are going to grade your student's essay."},
        {'role': 'user', 'content': instruction + essay},
        {'role': 'assistant', 'content': f'\n\nThe score is: '}
    ]
    
    count_try = 0
    while True:
        score = pipe(prompt, max_new_tokens=1, pad_token_id=pipe.tokenizer.eos_token_id)[0]['generated_text'][-1]['content']
        if score.isdigit():
            score = int(score)
            break
        else:
            count_try += 1
            
        if count_try >= 5:
            return None

    return score

In [6]:
submission = pd.read_csv(cfg.test_path)
submission['score'] = submission['full_text'].progress_map(prompt_engineering)
submission = submission.drop(columns = 'full_text')
submission.to_csv('submission.csv')
submission

100%|██████████| 3/3 [00:18<00:00,  6.29s/it]


,essay_id,score
0,000d118,3
1,000fe60,3
2,001ab80,4
